# LLM Session Memory

Large Language Models are inherently stateless and have no knowledge of previous interactions with a user, or even of previous parts of the current conversation. While this may not be noticable when asking simple questions, it becomes a hinderance when engaging in long running conversations that rely on conversational context.

The solution to this problem is to append the previous conversation history to each subsequent call to the LLM.

This notebook will show how to use Redis to structure and store and retrieve this conversational session memory.

In [1]:
from typing import Dict, List
import cohere
import os

class CohereClient():
    def __init__(self, api_key: str = None, model: str = 'command-r-plus'):
        api_key = api_key or os.getenv("COHERE_API_KEY")
        self.client = cohere.Client(api_key)
        self._model = model

    def converse(self, prompt: str, context: List[Dict]) -> str:
        context = self._remap(context)
        response = self.client.chat(
                model=self._model,
                chat_history = context,
                message=prompt,
                )
        return response.text


    def _remap(self, context) -> List[Dict]:
        ''' re-index the chat history to match the Cohere API requirements '''
        new_context = [] 
        for statement in context:
            if statement["role"] == "_user":
                new_statement = {"role": "USER", "message": statement["_content"]}
            elif statement["role"] == "_llm":
                new_statement = {"role": "CHATBOT", "message": statement["_content"]}
            elif statement["role"] == "_preamble":
                new_statement = {"role": "SYSTEM", "message": statement["_content"]}
            else:
                raise ValueError(f'Unknown chat role {statement["role"]}')
            new_context.append(new_statement)
        return new_context

/Users/justin.cechmanek/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [7]:
from redisvl.extensions.session_manager import SessionManager
user_session = SessionManager(name='llm_chef', session_id='123', user_id='abc', application_id='chefbot_123')
user_session.set_preamble("You are a helpful chef, assisting people in making delicious meals")

client = CohereClient()

On each call to the LLM we can first retrieve the recent history of the conversation. On the first call it will just be the preamble we set.

In [8]:
prompt = "can you give me some ideas for breakfast?"
context = user_session.conversation_history()
response = client.converse(prompt=prompt, context=context)
user_session.store((prompt, response))
print('USER: ', prompt)
print('\nLLM: ', response)

USER:  can you give me some ideas for breakfast?

LLM:  Sure! Here are a few breakfast ideas:

- Avocado toast: This is a simple yet delicious breakfast option. Just toast some bread, smash an avocado on top, and add a squeeze of lemon juice and a pinch of salt and pepper. You can also add some sliced tomato or a fried egg on top.
- Smoothie bowl: Blend together your favorite fruits and vegetables with a liquid base such as milk, yogurt, or juice to make a smoothie. Pour it into a bowl and top it with granola, nuts, seeds, and fresh fruit.
- Oatmeal: Cook some rolled oats in milk or water and top it with your favorite toppings such as fruit, nuts, seeds, cinnamon, or honey.
- Eggs: There are so many ways to enjoy eggs for breakfast! You can scramble them, fry them, poach them, or make an omelet with your favorite fillings. Serve them with some toast and fruit for a balanced meal.
- Pancakes or waffles: Make a batch of your favorite pancake or waffle batter and cook them up for a specia

Let's continue this conversation and inspect the context we're passing to the LLM as we go.

In [9]:
prompt = "can you give me the recipe for those pancakes?"
context = user_session.conversation_history()
response = client.converse(prompt=prompt, context=context)
user_session.store((prompt, response))
print('USER: ', prompt)
print('\nLLM: ', response)

USER:  can you give me the recipe for those pancakes?

LLM:  Sure, here's a basic recipe for pancakes that you can adjust and customize to your liking:

Ingredients:

1 1/2 cups all-purpose flour
3 1/2 teaspoons baking powder
1 teaspoon salt
1 tablespoon sugar
1 1/4 cups milk
1 egg
3 tablespoons melted butter, plus more for the pan

Instructions:

Whisk together the flour, baking powder, salt, and sugar in a large bowl.
In a separate bowl, whisk together the milk, egg, and melted butter.
Pour the wet ingredients into the dry ingredients and stir until just combined. Be careful not to overmix the batter.
Heat a nonstick pan or griddle over medium heat and grease it with a little butter.
Pour about 1/4 cup of batter onto the pan for each pancake. Cook until the edges look dry and there are bubbles on the surface, then flip and cook for another minute or two on the other side.
Serve the pancakes warm with your favorite toppings such as butter, syrup, fruit, or whipped cream.

Feel free to

In [10]:
for ctx in context:
    print(ctx)

{'role': '_preamble', '_content': 'You are a helpful chef, assisting people in making delicious meals'}
{'role': '_user', '_content': 'can you give me some ideas for breakfast?'}
{'role': '_llm', '_content': 'Sure! Here are a few breakfast ideas:\n\n- Avocado toast: This is a simple yet delicious breakfast option. Just toast some bread, smash an avocado on top, and add a squeeze of lemon juice and a pinch of salt and pepper. You can also add some sliced tomato or a fried egg on top.\n- Smoothie bowl: Blend together your favorite fruits and vegetables with a liquid base such as milk, yogurt, or juice to make a smoothie. Pour it into a bowl and top it with granola, nuts, seeds, and fresh fruit.\n- Oatmeal: Cook some rolled oats in milk or water and top it with your favorite toppings such as fruit, nuts, seeds, cinnamon, or honey.\n- Eggs: There are so many ways to enjoy eggs for breakfast! You can scramble them, fry them, poach them, or make an omelet with your favorite fillings. Serve t

In [11]:
prompt = "I am vegetarian. Can you remove the eggs?"
context = user_session.conversation_history()
response = client.converse(prompt=prompt, context=context)
user_session.store((prompt, response))
print('USER: ', prompt)
print('\nLLM: ', response)

USER:  I am vegetarian. Can you remove the eggs?

LLM:  Absolutely! Here's an egg-free version of the pancake recipe:

Ingredients:

1 1/2 cups all-purpose flour
3 1/2 teaspoons baking powder
1 teaspoon salt
1 tablespoon sugar
1 1/4 cups milk (dairy or non-dairy alternative)
3 tablespoons melted butter, plus more for the pan

Instructions:

Whisk together the flour, baking powder, salt, and sugar in a large bowl.
In a separate bowl, whisk together the milk and melted butter.
Pour the wet ingredients into the dry ingredients and stir until just combined. Be careful not to overmix the batter.
Heat a nonstick pan or griddle over medium heat and grease it with a little butter.
Pour about 1/4 cup of batter onto the pan for each pancake. Cook until the edges look dry and there are bubbles on the surface, then flip and cook for another minute or two on the other side.
Serve the pancakes warm with your favorite toppings such as butter, syrup, fruit, or whipped cream.

This version of the recip

In [12]:
for ctx in context:
    print(ctx)

{'role': '_preamble', '_content': 'You are a helpful chef, assisting people in making delicious meals'}
{'role': '_user', '_content': 'can you give me some ideas for breakfast?'}
{'role': '_llm', '_content': 'Sure! Here are a few breakfast ideas:\n\n- Avocado toast: This is a simple yet delicious breakfast option. Just toast some bread, smash an avocado on top, and add a squeeze of lemon juice and a pinch of salt and pepper. You can also add some sliced tomato or a fried egg on top.\n- Smoothie bowl: Blend together your favorite fruits and vegetables with a liquid base such as milk, yogurt, or juice to make a smoothie. Pour it into a bowl and top it with granola, nuts, seeds, and fresh fruit.\n- Oatmeal: Cook some rolled oats in milk or water and top it with your favorite toppings such as fruit, nuts, seeds, cinnamon, or honey.\n- Eggs: There are so many ways to enjoy eggs for breakfast! You can scramble them, fry them, poach them, or make an omelet with your favorite fillings. Serve t

In [14]:
prompt = "I am also vegan. Can you replace the butter too?"
context = user_session.conversation_history()
response = client.converse(prompt=prompt, context=context)
user_session.store((prompt, response))
print('USER: ', prompt)
print('\nLLM: ', response)

USER:  I am also vegan. Can you replace the butter too?

LLM:  Sure, to make the pancake recipe vegan, you can replace the butter with a vegan alternative. Here's the updated recipe:

Ingredients:

1 1/2 cups all-purpose flour
3 1/2 teaspoons baking powder
1 teaspoon salt
1 tablespoon sugar
1 1/4 cups plant-based milk (such as soy, almond, or oat milk)
3 tablespoons neutral-flavored oil (such as canola or vegetable oil), plus more for the pan

Instructions:

Whisk together the flour, baking powder, salt, and sugar in a large bowl.
In a separate bowl, whisk together the plant-based milk and oil.
Pour the wet ingredients into the dry ingredients and stir until just combined. Be careful not to overmix the batter.
Heat a nonstick pan or griddle over medium heat and grease it with a little oil.
Pour about 1/4 cup of batter onto the pan for each pancake. Cook until the edges look dry and there are bubbles on the surface, then flip and cook for another minute or two on the other side.
Serve t

In [15]:
for ctx in context:
    print(ctx)

{'role': '_preamble', '_content': 'You are a helpful chef, assisting people in making delicious meals'}
{'role': '_user', '_content': 'can you give me some ideas for breakfast?'}
{'role': '_llm', '_content': 'Sure! Here are a few breakfast ideas:\n\n- Avocado toast: This is a simple yet delicious breakfast option. Just toast some bread, smash an avocado on top, and add a squeeze of lemon juice and a pinch of salt and pepper. You can also add some sliced tomato or a fried egg on top.\n- Smoothie bowl: Blend together your favorite fruits and vegetables with a liquid base such as milk, yogurt, or juice to make a smoothie. Pour it into a bowl and top it with granola, nuts, seeds, and fresh fruit.\n- Oatmeal: Cook some rolled oats in milk or water and top it with your favorite toppings such as fruit, nuts, seeds, cinnamon, or honey.\n- Eggs: There are so many ways to enjoy eggs for breakfast! You can scramble them, fry them, poach them, or make an omelet with your favorite fillings. Serve t

This works, but our context keeps growing, increasing our LLM token count, which increases latency and cost.
Conversation histories can be truncated, but that that break quickly.

In [17]:
prompt = "I changed my mind. Can you give me the first recipe from your list?"
context = user_session.conversation_history(top_k=3)
response = client.converse(prompt=prompt, context=context)
user_session.store((prompt, response))
print('USER: ', prompt)
print('\nLLM: ', response)

USER:  I changed my mind. Can you give me the first recipe from your list?

LLM:  Sure, here's the first recipe for pancakes that I provided: 

Ingredients:

1 1/2 cups all-purpose flour
3 1/2 teaspoons baking powder
1 teaspoon salt
1 tablespoon sugar
1 1/4 cups milk
1 egg
3 tablespoons melted butter, plus more for the pan

Instructions:

Whisk together the flour, baking powder, salt, and sugar in a large bowl.
In a separate bowl, whisk together the milk, egg, and melted butter.
Pour the wet ingredients into the dry ingredients and stir until just combined. Be careful not to overmix the batter; a few lumps are okay.
Heat a nonstick pan or griddle over medium heat and grease it with a little butter.
Pour about 1/4 cup of batter onto the pan for each pancake. Cook until the edges look dry and small bubbles start to appear on the surface, about 2-3 minutes.
Flip the pancakes carefully and cook for an additional 1-2 minutes on the other side until golden brown.
Serve the pancakes warm with

In [18]:
for ctx in context:
    print(ctx)

{'role': '_preamble', '_content': 'You are a helpful chef, assisting people in making delicious meals'}
{'role': '_user', '_content': 'can you give me the recipe for those pancakes?'}
{'role': '_llm', '_content': "Sure, here's a basic recipe for pancakes that you can adjust and customize to your liking:\n\nIngredients:\n\n1 1/2 cups all-purpose flour\n3 1/2 teaspoons baking powder\n1 teaspoon salt\n1 tablespoon sugar\n1 1/4 cups milk\n1 egg\n3 tablespoons melted butter, plus more for the pan\n\nInstructions:\n\nWhisk together the flour, baking powder, salt, and sugar in a large bowl.\nIn a separate bowl, whisk together the milk, egg, and melted butter.\nPour the wet ingredients into the dry ingredients and stir until just combined. Be careful not to overmix the batter.\nHeat a nonstick pan or griddle over medium heat and grease it with a little butter.\nPour about 1/4 cup of batter onto the pan for each pancake. Cook until the edges look dry and there are bubbles on the surface, then f

The LLM has forgotten about the avocado toast it mentioned because that was too far back in it's conversation history!

## Semantic session memory

Rather than naively give pass the most recent conversation we can use RedisVL to find only the semantically relevant sections of our conversation based on our new prompt. We can do this with the `fetch_context()` method.

In [52]:
prompt = "Can you give me the avocado one?"
user_session.set_distance_threshold(0.75)
context = user_session.fetch_context(prompt=prompt)
response = client.converse(prompt=prompt, context=context)
user_session.store((prompt, response))
print('USER: ', prompt)
print('\nLLM: ', response)

USER:  Can you give me the avocado one?

LLM:  Sure, here's a simple recipe for avocado toast:

Ingredients:
- 1 ripe avocado
- 1 slice of your favorite bread
- Lemon juice, to taste
- Salt and pepper, to taste
- Optional: sliced tomato, a fried egg, or any other desired toppings

Directions:
1. Toast your bread to your desired level.
2. While the bread is toasting, smash the avocado in a bowl with a fork until it reaches your desired consistency. You can leave it chunkier or make it smoother, depending on your preference.
3. Add a squeeze of lemon juice to the avocado and season with salt and pepper to taste. Mix it all together.
4. Once the bread is toasted, spread the avocado mixture on top.
5. If desired, add any additional toppings such as sliced tomato or a fried egg.
6. Enjoy your delicious and nutritious avocado toast!

Feel free to experiment with different types of bread and additional toppings to create your own unique version of avocado toast!


In [53]:
for ctx in context:
    print(ctx)

{'role': '_preamble', '_content': 'You are a helpful chef, assisting people in making delicious meals'}
{'role': '_user', '_content': 'can you give me some ideas for breakfast?'}
{'role': '_llm', '_content': 'Sure! Here are a few breakfast ideas:\n\n- Avocado toast: This is a simple yet delicious breakfast option. Just toast some bread, smash an avocado on top, and add a squeeze of lemon juice and a pinch of salt and pepper. You can also add some sliced tomato or a fried egg on top.\n- Smoothie bowl: Blend together your favorite fruits and vegetables with a liquid base such as milk, yogurt, or juice to make a smoothie. Pour it into a bowl and top it with granola, nuts, seeds, and fresh fruit.\n- Oatmeal: Cook some rolled oats in milk or water and top it with your favorite toppings such as fruit, nuts, seeds, cinnamon, or honey.\n- Eggs: There are so many ways to enjoy eggs for breakfast! You can scramble them, fry them, poach them, or make an omelet with your favorite fillings. Serve t

Looking at the context we see that only the conversation section that had to do with avocados was retrieved. We don't need to know about pancakes to make avocado toast.

###############################
Repeat with OpenAI
###############################

Start by creating a simple wrapper around your LLM provider of choice. Here we'll use [OpenAI](https://platform.openai.com).

In [18]:
from typing import Dict, List
import os
from openai import OpenAI

class OpenAIClient():
    def __init__(self, api_key: str = None, model:str ='gpt-3.5-turbo'):
        api_key = api_key or os.getenv("OPENAI_API_KEY")
        self.client = OpenAI(api_key=api_key)
        self._model = model

    def converse(self, prompt: str, context: List[Dict], max_tokens: int = 250) -> str:
        context = self._remap(context)
        messages = context + [{"role": "user", "content": prompt}]
        response = self.client.chat.completions.create(
            model = self._model,
            messages = messages,
            max_tokens = max_tokens
        )
        return response.choices[0].message.content

    def _remap(self, context: List[Dict]) -> List[Dict]:
        ''' re-index the chat history to match the OpenAI API requirements '''
        new_context = []
        for statement in context:
            if statement["role"] == "_user":
                new_statement = {"role": "user", "content": statement["_content"]}
            elif statement["role"] == "_llm":
                new_statement = {"role": "assistant", "content": statement["_content"]}
            elif statement["role"] == "_preamble":
                new_statement = {"role": "system", "content": statement["_content"]}
            else:
                raise ValueError(f'Unknown chat role {statement["role"]}')
            new_context.append(new_statement)
        return new_context

To store and retrieve our conversational history we'll use a `SessionManager` from RedisVL.
As an example, we'll have OpenAI help with our meal planning. We'll set the preamble - a permanent prompt that will be included at the start of every LLM call to tell the model how to behave.

In [17]:
from redisvl.extensions.session_manager import SessionManager

In [19]:
user_session = SessionManager(name='example', session_id='123', user_id='abc', application_id='sample_app')
user_session.set_preamble("You are a helpful chef, assisting people in making delicious meals")

client = OpenAIClient()

10:34:35 redisvl.index.index INFO   Index already exists, not overwriting.


In [20]:
prompt = "can you give me some ideas for breakfast?"
context = user_session.fetch_context(prompt)
response = client.converse(prompt=prompt, context=context)
user_session.store((prompt, response))
print('USER: ', prompt)
print('\nLLM: ', response)

USER:  can you give me some ideas for breakfast?

LLM:  Of course! Here are some breakfast ideas for you:

1. Classic French toast with a dusting of powdered sugar and a drizzle of maple syrup.
2. Homemade pancakes topped with fresh berries, a dollop of Greek yogurt, and a drizzle of honey.
3. Breakfast parfait with layers of Greek yogurt, homemade granola, and mixed berries.
4. Veggie frittata made with eggs, spinach, cherry tomatoes, and feta cheese.
5. Chia seed pudding topped with sliced bananas, almonds, and a sprinkle of cinnamon.
6. Breakfast quesadilla filled with scrambled eggs, black beans, cheese, and salsa.
7. Overnight oats with almond milk, chia seeds, sliced almonds, and diced mango.

I hope you find these breakfast ideas inspiring and delicious! Enjoy your morning meal.


In [21]:
prompt = "can you give me the recipe for those pancakes?"
context = user_session.fetch_context(prompt)
response = client.converse(prompt=prompt, context=context)
user_session.store((prompt, response))
print('USER: ', prompt)
print('\nLLM: ', response)

USER:  can you give me the recipe for those pancakes?

LLM:  Absolutely! Here's a simple and delicious pancake recipe for you to enjoy:

Ingredients:
- 1 cup all-purpose flour
- 2 tablespoons granulated sugar
- 1 teaspoon baking powder
- 1/2 teaspoon baking soda
- 1/4 teaspoon salt
- 1 cup buttermilk
- 1 large egg
- 2 tablespoons melted butter
- 1 teaspoon vanilla extract
- Butter or oil for greasing the pan

Instructions:
1. In a large mixing bowl, whisk together the flour, sugar, baking powder, baking soda, and salt.
2. In a separate bowl, whisk together the buttermilk, egg, melted butter, and vanilla extract.
3. Pour the wet ingredients into the dry ingredients and gently stir until just combined. Be careful not to overmix; a few lumps in the batter are okay.
4. Let the batter rest for about 5 minutes while you preheat a non-stick skillet or griddle over medium heat.
5. Grease the pan with a little butter or oil. Use a 1/4 cup measuring cup to pour the batter onto the pan.
6. Cook t

In [22]:
for ctx in context:
    print(ctx)

{'role': '_preamble', '_content': 'You are a helpful chef, assisting people in making delicious meals'}
{'role': '_user', '_content': 'I changed my mind. Give me the veggie omlette recipe.'}
{'role': '_llm', '_content': "Absolutely! Here's a simple and delicious veggie omelette recipe for you:\n\nIngredients:\n- 3 large eggs\n- 1/4 cup sliced mushrooms\n- 1/4 cup diced bell peppers (any color you prefer)\n- 1 cup fresh spinach leaves\n- 1/4 cup crumbled feta cheese\n- Salt and pepper to taste\n- Olive oil for cooking\n\nInstructions:\n1. In a bowl, whisk the eggs together until well beaten. Season with a pinch of salt and pepper.\n2. Heat a non-stick skillet over medium heat and add a drizzle of olive oil.\n3. Add the mushrooms and bell peppers to the skillet and sauté for a few minutes until they start to soften.\n4. Add the spinach leaves to the skillet and cook until wilted.\n5. Pour the beaten eggs over the vegetables in the skillet. Tilt the skillet to spread the eggs evenly.\n6. 

In [23]:
prompt = "I'm vegan. Can you remove the eggs and update that recipe?"
context = user_session.fetch_context(prompt)
response = client.converse(prompt=prompt, context=context)
user_session.store((prompt, response))
print('USER: ', prompt)
print('LLM: ', response)

USER:  I'm vegan. Can you remove the eggs and update that recipe?
LLM:  Absolutely! Here's a vegan version of the pancake recipe for you:

Ingredients:
- 1 cup all-purpose flour
- 2 tablespoons granulated sugar
- 1 tablespoon baking powder
- 1/4 teaspoon salt
- 1 cup non-dairy milk (such as almond or soy milk)
- 2 tablespoons melted coconut oil or vegetable oil
- 1 teaspoon vanilla extract

Instructions:
1. In a mixing bowl, whisk together the flour, sugar, baking powder, and salt.
2. In a separate bowl, whisk together the non-dairy milk, melted coconut oil, and vanilla extract.
3. Pour the wet ingredients into the dry ingredients and mix until just combined. Be careful not to overmix; a few lumps in the batter are okay.
4. Let the batter rest for about 5 minutes while you preheat a non-stick skillet or griddle over medium heat.
5. Grease the pan with a little coconut oil or vegetable oil. Use a 1/4 cup measuring cup to pour the batter onto the pan.
6. Cook the pancakes for 2-3 minutes

In [24]:
for ctx in context:
    print(ctx)

{'role': '_preamble', '_content': 'You are a helpful chef, assisting people in making delicious meals'}
{'role': '_user', '_content': 'Can you take out the feta cheese from that?'}
{'role': '_llm', '_content': "Of course! Here's the modified veggie omelette recipe without feta cheese:\n\nIngredients:\n- 3 large eggs\n- 1/4 cup sliced mushrooms\n- 1/4 cup diced bell peppers (any color you prefer)\n- 1 cup fresh spinach leaves\n- Salt and pepper to taste\n- Olive oil for cooking\n\nInstructions:\n1. In a bowl, whisk the eggs together until well beaten. Season with a pinch of salt and pepper.\n2. Heat a non-stick skillet over medium heat and add a drizzle of olive oil.\n3. Add the mushrooms and bell peppers to the skillet and sauté for a few minutes until they start to soften.\n4. Add the spinach leaves to the skillet and cook until wilted.\n5. Pour the beaten eggs over the vegetables in the skillet. Tilt the skillet to spread the eggs evenly.\n6. Cook the omelette for a few minutes until

In [25]:
prompt = "Can you swap out the bananas for blue berries?"
context = user_session.fetch_context(prompt)
response = client.converse(prompt=prompt, context=context)
user_session.store((prompt, response))
print('USER: ', prompt)
print('LLM: ', response)

USER:  Can you swap out the bananas for blue berries?
LLM:  Of course! You can easily make blueberry pancakes by adding fresh or frozen blueberries to the pancake batter. Here's how you can modify the vegan pancake recipe to include blueberries:

Ingredients:
- 1 cup all-purpose flour
- 2 tablespoons granulated sugar
- 1 tablespoon baking powder
- 1/4 teaspoon salt
- 1 cup non-dairy milk (such as almond or soy milk)
- 2 tablespoons melted coconut oil or vegetable oil
- 1 teaspoon vanilla extract
- 1/2 cup fresh or frozen blueberries

Instructions:
1. Follow the same steps as the vegan pancake recipe above until you have mixed the wet and dry ingredients together.
2. Gently fold in the blueberries into the pancake batter until evenly distributed.
3. Let the batter rest for about 5 minutes while you preheat a non-stick skillet or griddle over medium heat.
4. Grease the pan with a little coconut oil or vegetable oil. Use a 1/4 cup measuring cup to pour the blueberry pancake batter onto th

In [26]:
for ctx in context:
    print(ctx)

{'role': '_preamble', '_content': 'You are a helpful chef, assisting people in making delicious meals'}
{'role': '_user', '_content': 'can you give me some ideas for breakfast?'}
{'role': '_llm', '_content': 'Of course! Here are some breakfast ideas for you:\n\n1. Classic French toast with a dusting of powdered sugar and a drizzle of maple syrup.\n2. Homemade pancakes topped with fresh berries, a dollop of Greek yogurt, and a drizzle of honey.\n3. Breakfast parfait with layers of Greek yogurt, homemade granola, and mixed berries.\n4. Veggie frittata made with eggs, spinach, cherry tomatoes, and feta cheese.\n5. Chia seed pudding topped with sliced bananas, almonds, and a sprinkle of cinnamon.\n6. Breakfast quesadilla filled with scrambled eggs, black beans, cheese, and salsa.\n7. Overnight oats with almond milk, chia seeds, sliced almonds, and diced mango.\n\nI hope you find these breakfast ideas inspiring and delicious! Enjoy your morning meal.'}
{'role': '_user', '_content': 'can you

In [27]:

prompt = "I changed my mind. Give me the yogurt parfait recipe."
context = user_session.fetch_context(prompt)
response = client.converse(prompt=prompt, context=context)
user_session.store((prompt, response))
print('USER: ', prompt)
print('LLM: ', response)

USER:  I changed my mind. Give me the yogurt parfait recipe.
LLM:  Of course! Here's a delicious and simple recipe for a yogurt parfait:

Ingredients:
- 1 cup Greek yogurt (or your favorite non-dairy alternative)
- 1/2 cup granola
- 1/2 cup mixed berries (such as strawberries, blueberries, raspberries)
- 1 tablespoon honey or maple syrup (optional)
- A sprinkle of cinnamon (optional)

Instructions:
1. In a glass or bowl, start with a layer of Greek yogurt at the bottom.
2. Add a layer of granola on top of the yogurt.
3. Add a layer of mixed berries on top of the granola.
4. Repeat the layers until you've used up all the ingredients, finishing with a final sprinkle of granola on top.
5. Drizzle honey or maple syrup on top for extra sweetness, if desired.
6. Add a sprinkle of cinnamon for extra flavor, if desired.

Enjoy your delicious and nutritious yogurt parfait! You can also customize this recipe by adding additional toppings like nuts, seeds, or dried fruit.


In [28]:
for ctx in context:
    print(ctx)

{'role': '_preamble', '_content': 'You are a helpful chef, assisting people in making delicious meals'}
{'role': '_user', '_content': 'can you give me the recipe for those pancakes?'}
{'role': '_llm', '_content': "Absolutely! Here's a simple and delicious pancake recipe for you to enjoy:\n\nIngredients:\n- 1 cup all-purpose flour\n- 2 tablespoons granulated sugar\n- 1 teaspoon baking powder\n- 1/2 teaspoon baking soda\n- 1/4 teaspoon salt\n- 1 cup buttermilk\n- 1 large egg\n- 2 tablespoons melted butter\n- 1 teaspoon vanilla extract\n- Butter or oil for greasing the pan\n\nInstructions:\n1. In a large mixing bowl, whisk together the flour, sugar, baking powder, baking soda, and salt.\n2. In a separate bowl, whisk together the buttermilk, egg, melted butter, and vanilla extract.\n3. Pour the wet ingredients into the dry ingredients and gently stir until just combined. Be careful not to overmix; a few lumps in the batter are okay.\n4. Let the batter rest for about 5 minutes while you pre

In [29]:
prompt = "What are some other recipes that use Greek yogurt?"
context = user_session.fetch_context(prompt)
response = client.converse(prompt=prompt, context=context)
user_session.store((prompt, response))
print('USER: ', prompt)
print('LLM: ', response)

USER:  What are some other recipes that use Greek yogurt?
LLM:  Greek yogurt is a versatile ingredient that can be used in many different recipes, both sweet and savory. Here are a few ideas for dishes that feature Greek yogurt:

1. Tzatziki Sauce: A classic Greek sauce made with Greek yogurt, cucumber, garlic, lemon juice, and dill. It's perfect as a dip for veggies, pita bread, or as a sauce for grilled meats.

2. Greek Yogurt Pancakes: Use Greek yogurt in your pancake batter for a fluffy and protein-packed breakfast. You can also add fruits or nuts to customize the flavor.

3. Greek Yogurt Parfait: As mentioned earlier, layer Greek yogurt with granola and fresh fruit for a simple and delicious parfait.

4. Greek Yogurt Lemon Chicken: Marinate chicken breasts in a mixture of Greek yogurt, lemon juice, garlic, and herbs before grilling or baking for a tender and flavorful main dish.

5. Greek Yogurt Salad Dressing: Blend Greek yogurt with herbs, lemon juice, and olive oil to make a cr

In [30]:
for ctx in context:
    print(ctx)

{'role': '_preamble', '_content': 'You are a helpful chef, assisting people in making delicious meals'}
{'role': '_user', '_content': "I'm vegan. Can you remove the eggs and update that recipe?"}
{'role': '_llm', '_content': "Absolutely! Here's a vegan version of the pancake recipe for you:\n\nIngredients:\n- 1 cup all-purpose flour\n- 2 tablespoons granulated sugar\n- 1 tablespoon baking powder\n- 1/4 teaspoon salt\n- 1 cup non-dairy milk (such as almond or soy milk)\n- 2 tablespoons melted coconut oil or vegetable oil\n- 1 teaspoon vanilla extract\n\nInstructions:\n1. In a mixing bowl, whisk together the flour, sugar, baking powder, and salt.\n2. In a separate bowl, whisk together the non-dairy milk, melted coconut oil, and vanilla extract.\n3. Pour the wet ingredients into the dry ingredients and mix until just combined. Be careful not to overmix; a few lumps in the batter are okay.\n4. Let the batter rest for about 5 minutes while you preheat a non-stick skillet or griddle over med

In [31]:
prompt = "I changed my mind again. Give me back the yogurt parfait recipe."
context = user_session.fetch_context(prompt)
response = client.converse(prompt=prompt, context=context)
user_session.store((prompt, response))
print('USER: ', prompt)
print('LLM: ', response)

USER:  I changed my mind again. Give me back the yogurt parfait recipe.
LLM:  Of course! Here's the yogurt parfait recipe again:

Ingredients:
- 1 cup Greek yogurt (or your favorite non-dairy alternative)
- 1/2 cup granola
- 1/2 cup mixed berries (such as strawberries, blueberries, raspberries)
- 1 tablespoon honey or maple syrup (optional)
- A sprinkle of cinnamon (optional)

Instructions:
1. In a glass or bowl, start with a layer of Greek yogurt at the bottom.
2. Add a layer of granola on top of the yogurt.
3. Add a layer of mixed berries on top of the granola.
4. Repeat the layers until you've used up all the ingredients, finishing with a final sprinkle of granola on top.
5. Drizzle honey or maple syrup on top for extra sweetness, if desired.
6. Add a sprinkle of cinnamon for extra flavor, if desired.

Enjoy your delicious and nutritious yogurt parfait! Feel free to customize it with your favorite toppings or ingredients to make it your own.


In [32]:
for ctx in context:
    print(ctx)

{'role': '_preamble', '_content': 'You are a helpful chef, assisting people in making delicious meals'}
{'role': '_user', '_content': 'I changed my mind. Give me the yogurt parfait recipe.'}
{'role': '_llm', '_content': "Of course! Here's a delicious and simple recipe for a yogurt parfait:\n\nIngredients:\n- 1 cup Greek yogurt (or your favorite non-dairy alternative)\n- 1/2 cup granola\n- 1/2 cup mixed berries (such as strawberries, blueberries, raspberries)\n- 1 tablespoon honey or maple syrup (optional)\n- A sprinkle of cinnamon (optional)\n\nInstructions:\n1. In a glass or bowl, start with a layer of Greek yogurt at the bottom.\n2. Add a layer of granola on top of the yogurt.\n3. Add a layer of mixed berries on top of the granola.\n4. Repeat the layers until you've used up all the ingredients, finishing with a final sprinkle of granola on top.\n5. Drizzle honey or maple syrup on top for extra sweetness, if desired.\n6. Add a sprinkle of cinnamon for extra flavor, if desired.\n\nEnjo